In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import os
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Load Data

In [ ]:
df = pd.read_csv("/kaggle/input/turkish-poems/all_poems.csv")
df.head()

# Utils

In [ ]:
poems = ''.join(poem_text for poem_text in df.poem_text.values)

In [ ]:
len(poems)

In [ ]:
chars = ''.join(sorted(set(poems)))

In [ ]:
chars

In [ ]:
def char_to_int(word):
    indices = [chars.index(char) for char in word]
    tensor = torch.tensor(indices, dtype=torch.long)
    return torch.autograd.Variable(tensor)

# Model

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=1)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, inputs, hidden):
        inputs = self.encoder(inputs.view(1, -1))
        output, hidden = self.lstm(inputs.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_size),
                torch.zeros(1, 1, self.hidden_size))

In [ ]:
input_size = len(chars)
output_size = len(chars)
hidden_size = 100

In [ ]:
model = LSTM(len(chars), hidden_size, output_size)

# Train

In [ ]:
lr = 0.005
chunk_len = 500

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
losses = []

In [ ]:
n_epochs = 2000

In [ ]:
for epoch in range(n_epochs):
    start_index = random.randint(0, len(poems) - chunk_len)
    end_index = start_index + chunk_len + 1
    chunk = poems[start_index:end_index]

    inputs = char_to_int(chunk[:-1])
    target = char_to_int(chunk[1:])

    hidden, cell = model.init_hidden()
    model.zero_grad()

    loss = 0
    for char in range(chunk_len):
        output, (hidden, cell) = model(inputs[char], (hidden, cell))
        loss += criterion(output, target[char].unsqueeze(0))

    loss.backward()
    optimizer.step()

    avg = loss.item() / chunk_len
    losses.append(avg)

    if epoch % 100 == 0:
        print(f"Epoch [{epoch}/{n_epochs}]: Loss: {avg:.4f}")

# Results

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(losses)
plt.title("Loss Curve")
plt.legend(["Train Loss"])
plt.show()

In [ ]:
def generate_poems(first_text, temperature):
    hidden, cell = model.init_hidden()
    first_text_int = char_to_int(first_text)
    predicted = first_text

    for char in range(len(first_text) - 1):
        _, (hidden, cell) = model(first_text_int[char], (hidden, cell))

    inputs = first_text_int[-1]

    for _ in range(chunk_len):
        output, (hidden, cell) = model(inputs, (hidden, cell))
        output = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output, 1)[0].item()

        predicted_char = chars[top_i]
        predicted += predicted_char
        inputs = char_to_int(predicted_char)

    return predicted

In [ ]:
print(generate_poems("Al", 0.2))